In [3]:
import requests
import bs4
import os
import pandas as pd
import json

#### 取得最大頁面

In [18]:
url = "https://www.4p.com.tw/EW/GO/GroupList.asp"

In [19]:
html = requests.get(url)
soup = bs4.BeautifulSoup(html.text, "lxml")
max_page = int(soup.find("input", attrs={"name":"maxPageGo"})["value"])
print("目前可爬取最多頁面:", max_page)

目前可爬取最多頁面: 80


#### 可設定爬取頁面資料的其他方法

url = "https://www.newamazing.com.tw/EW/Services/SearchListData.asp"

page = 1  
beginDt = "2020/06/23"  
endDt = "2021/06/23"  

data = {"pageAll": page,"beginDt": beginDt,"endDt":endDt}  
html = requests.get(url, data=data)  
soup = bs4.BeautifulSoup(html.text, "lxml")  

#### 爬取行程編號並清洗

In [20]:
def getGruopNumber(url, start, end=None):
    
    page = start
    data = {"pageAll": page} # 可擴充搜尋選項
    
    groupNumber = []
    
    if(end==None):
        html = requests.get(url, data=data)
        soup = bs4.BeautifulSoup(html.text, "lxml")
        
        for i in soup.find_all("span", "product_num"):
            groupNumber.append(i.text)
    else:
        for i in range(start, end):
            data["pageAll"] = i
            html = requests.get(url, data=data)
            soup = bs4.BeautifulSoup(html.text, "lxml")
        
            for j in soup.find_all("span", "product_num"):
                groupNumber.append(j.text)
                
    return groupNumber

In [21]:
groupNumbers = getGruopNumber(url, 1, 10)

In [22]:
len(groupNumbers)

324

#### 爬取行程頁面

In [28]:
def getTrip(groupNumber):
    
    url = "https://www.4p.com.tw/EW/GO/GroupDetail.asp?prodCd=" + groupNumber

    html = requests.get(url)
    soup = bs4.BeautifulSoup(html.text, "lxml")
    
    # 標題
    title = soup.find("h3").text
    travel_agency = soup.find("title").text.replace(title+"-","")
    
    # 價錢
    price = ""
    # 處理國外旅團和國內的差異
    if(soup.find("li", "col-xs-12 col-sm-12 col-md-2 col-lg-2 price_content")==None):
        for i in soup.find("div", "panel-body").find("span").text:
            if(i.isnumeric()):
                price+=i
    else:
        for i in soup.find("li", "col-xs-12 col-sm-12 col-md-2 col-lg-2 price_content").text:
            if(i.isnumeric()):
                price+=i

    days = soup.find("li", "col-xs-12 col-sm-12 col-md-2 col-lg-2 return_date").text.replace(" ","").replace("\r\n","")
    
    # 出團日期 & 天數
    dates = []
    remaining_number = []
    for month in soup.find_all("div", attrs={"role":"tabpanel"}):
        for date in month.find_all("ul", "list_content"):
            dates.append(date.find("li").text)
            remaining_number.append(int(date.find("span","badge").text))
    try:
        # 取出圖片網址
        imgs = []
        imgs_index = []
        for img in soup.find("table").find_all("img"):
            imgs.append(img["src"])
        # 下載圖片並儲存
        for img in imgs:
            folder_path = "agency/NEWAMAZE/imgs"
            html = requests.get(img)

            name = img.replace("https://www.4p.com.tw/eWeb_spunktour/IMGDB/", "")
            # https://www.newamazing.com.tw/eweb_newamazing/newfile2013/
            temp = name.split("/")

            for i in range(len(temp)-1):
                folder_path = folder_path + "/" + temp[i]
                if (os.path.exists(folder_path) == False): #判斷資料夾是否存在
                    os.makedirs(folder_path) #Create folder

            img_name = temp[-1]
            imgs_index.append(folder_path + "/" + img_name)

            if (img_name not in os.listdir(folder_path)):
                with open(folder_path + "/" + img_name,'wb') as file: #以byte的形式將圖片數據寫入
                    file.write(html.content)
    except Exception as e:
        print("沒有額外圖片")
        
    # 航班資料
    flights = []
    for flight in soup.find_all("ul","flight_content"):
        temp = []
        for detail in flight.find_all("li"):
            temp.append(detail.text)
        flights.append(temp)
                
    return {"itinerary":groupNumber,"title":title,"travel_agency":travel_agency,"price":price,"days":days,"dates":dates,"remaining_number":remaining_number,"flights":flights}

### 爬蟲解析方式開發過程

In [30]:
groupNumber = groupNumbers[0]

In [31]:
dict = getTrip(groupNumber)

沒有額外圖片


In [ ]:
title = dict["title"]
travel_agency = dict["travel_agency"].replace(title, "").replace(" ","").split("-")[1]
price = dict["price"]
days = dict["days"]
dict["date"] = dict["travel_agency"].replace(title, "").replace(" ","").split("-")[0]

In [584]:
# 處理團細節
csv_groupDetail = pd.DataFrame({"groupNumber":[dict["itinerary"]],"title":[title],"travel_agency":[travel_agency],"price":[price],"days":[days]})

In [585]:
# 處理日期和剩多少團位
temp = {"dates":dict["dates"], "remaining_number":dict["remaining_number"]}

csv_groupRemaining = pd.DataFrame(temp)

In [586]:
# 處理班機資訊
columns = ["groupNumber","d_day","d_airlane","d_number","d_departure","d_time","d_destination","d_time2",
           "b_day","b_airlane","b_number","b_departure","b_time","b_destination","b_time2"]
csv_flights =  pd.DataFrame(columns=columns)

if(dict["flights"]==[]):
    csv_flights = []
else:
    temp = {}
    temp2 = dict["flights"][0] + dict["flights"][1]
    temp2.insert(0, groupNumber)
    for i in range(len(temp2)):
        temp.update({columns[i]:temp2[i]})
    csv_flights = csv_flights.append(temp,ignore_index=True)

In [592]:
path = "agency/NEWAMAZE/"
csv_groupDetail.to_csv(path + "groupDetail.csv", header=False, index=False, mode="a")

path = "agency/NEWAMAZE/dateRemaining/"
csv_groupRemaining.to_csv(path + dict["itinerary"]+".csv")


if(dict["flights"]==[]):
    pass
else:
    path = "agency/NEWAMAZE/"
    csv_flights.to_csv(path + "flights.csv", header=False, index=False, mode="a")

### 行程詳細介紹略過

#### 大量爬蟲

In [29]:
for i in range(len(groupNumbers)):
    try:
        groupNumber = groupNumbers[i]
        dict = getTrip(groupNumber)

        title = dict["title"]
        travel_agency = dict["travel_agency"].replace(title, "").replace(" ","").split("-")[1]
        price = dict["price"]
        days = dict["days"]
        dict["date"] = dict["travel_agency"].replace(title, "").replace(" ","").split("-")[0]

        # 處理團細節
        csv_groupDetail = pd.DataFrame({"groupNumber":[dict["itinerary"]],"title":[title],"travel_agency":[travel_agency],"price":[price],"days":[days]})

        # 處理日期和剩多少團位
        temp = {"dates":dict["dates"], "remaining_number":dict["remaining_number"]}

        csv_groupRemaining = pd.DataFrame(temp)

        # 處理班機資訊
        columns = ["groupNumber","d_day","d_airlane","d_number","d_departure","d_time","d_destination","d_time2",
                   "b_day","b_airlane","b_number","b_departure","b_time","b_destination","b_time2"]
        csv_flights =  pd.DataFrame(columns=columns)

        if(dict["flights"]==[]):
            csv_flights = []
        else:
            temp = {}
            temp2 = dict["flights"][0] + dict["flights"][1]
            temp2.insert(0, groupNumber)
            for i in range(len(temp2)):
                temp.update({columns[i]:temp2[i]})
            csv_flights = csv_flights.append(temp,ignore_index=True)

        path = "agency/NEWAMAZE/"
        csv_groupDetail.to_csv(path + "groupDetail.csv", header=False, index=False, mode="a")

        path = "agency/NEWAMAZE/dateRemaining/"
        csv_groupRemaining.to_csv(path + dict["itinerary"]+".csv")


        if(dict["flights"]==[]):
            pass
        else:
            path = "agency/NEWAMAZE/"
            csv_flights.to_csv(path + "flights.csv", header=False, index=False, mode="a")
    except Exception as e:
        print(e)
        print("error number: ", i)

沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
'NoneType' object has no attribute 'text'
error number:  24
'NoneType' object has no attribute 'text'
error number:  25
'NoneType' object has no attribute 'text'
error number:  26
'NoneType' object has no attribute 'text'
error number:  27
'NoneType' object has no attribute 'text'
error number:  28
'NoneType' object has no attribute 'text'
error number:  29
'NoneType' object has no attribute 'text'
error number:  30
'NoneType' object has no attribute 'text'
error number:  31
'NoneType' object has no attribute 'text'
error number:  32
'NoneType' object has no attribute 'text'
error number:  33
'NoneType' object has no attribute 'text'
error number:  34
'NoneType' object has no attribute 'text'
error number:  35
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
沒有額外圖片
